In [78]:
using CSV, DataFrames
using PhysicalConstants

df = CSV.read("prem.csv", header = true, delim = ",", DataFrame)
premDensity = Matrix(df[:, [1, 3]])
premDensityMatrixSize = size(premDensity, 1) # *** Raw data in g/cm³
display(premDensity)

197×2 Matrix{Float64}:
 6369.0   1.02
 6368.0   1.02
 6368.0   2.6
 6367.0   2.6
 6366.0   2.6
 6365.0   2.6
 6364.0   2.6
 6363.0   2.6
 6362.0   2.6
 6361.0   2.6
    ⋮    
  800.0  12.9491
  700.0  12.9818
  600.0  13.0101
  500.0  13.0341
  400.0  13.0537
  300.0  13.0689
  200.0  13.0798
  100.0  13.0863
    0.0  13.0885

In [80]:
# Transforms the matrix into shell mass which should be more calculable

premShellGravity = [0.0 0.0]
G = 6.67e-2 # In km³kg⁻¹s⁻²
for i in range(2, premDensityMatrixSize)
    global premDensity
    outerShellRadius = premDensity[i - 1, 1] # Calculating in radius, not in depths
    innerShellRadius = premDensity[i, 1]
    shellDensity = premDensity[i, 2] # Convert from g/cm³ to kg/m³
    shellGravity = G*shellDensity*4/3*π*(outerShellRadius^3 - innerShellRadius^3)
    premShellGravity = [premShellMass; outerShellRadius shellGravity]
end

show(stdout, "text/plain", premShellGravity)

196×2 Matrix{Float64}:
    0.0  0.0
 6370.0  5.20184e8
 6369.0  5.20021e8
 6368.0  5.19858e8
 6368.0  0.0
 6367.0  1.32471e9
 6366.0  1.32429e9
 6365.0  1.32388e9
 6364.0  1.32346e9
 6363.0  1.32305e9
 6362.0  1.32263e9
 6361.0  1.32222e9
 6360.0  1.3218e9
 6359.0  1.32138e9
 6358.0  1.32097e9
 6357.0  1.32055e9
 6356.0  1.32014e9
 6356.0  0.0
 6355.0  1.472e9
 6354.0  1.47154e9
 6353.0  1.47107e9
 6352.0  1.47061e9
 6351.0  1.47015e9
 6350.0  1.46968e9
 6349.0  1.46922e9
 6348.0  1.46876e9
 6346.6  2.05548e9
 6346.6  0.0
 6346.0  1.02661e9
 6345.0  1.71053e9
 6344.0  1.70994e9
 6343.0  1.70935e9
 6342.0  1.70875e9
 6341.0  1.70816e9
 6340.0  1.70757e9
 6339.0  1.70697e9
 6338.0  1.70638e9
 6337.0  1.70578e9
 6336.0  1.70519e9
 6331.0  8.51652e9
 6326.0  8.50172e9
 6321.0  8.48693e9
 6311.0  1.69282e10
 6301.0  1.68692e10
 6291.0  1.68103e10
 6291.0  0.0
 6281.0  1.67515e10
 6271.0  1.66929e10
 6261.0  1.66344e10
 6251.0  1.6576e10
 6241.0  1.65177e10
 6231.0  1.64595e10
 6221.0  1.640

In [87]:
# Clearing all zeros to prevent divide by zero problems

### Fix this
zerosRow = .!any(premShellGravity .== 0, dims = 2)
tempMatrix = Matrix{Float64}(undef, 2, 2)
for i in size(premShellGravity, 1)
    if zerosRow[i] != 0
        tempMatrix = [tempMatrix; premShellGravity[i, 1] premShellGravity[i, 2]]
    end
end
display(tempMatrix)

2×2 Matrix{Float64}:
 6.90923e-310  6.90918e-310
 6.90923e-310  6.90918e-310

In [76]:
# Calculating the total gravitational force from the object's position.

objectPos = 6390 # In km
netAccel = 0
G = 6.67e-2 # In km³kg⁻¹s⁻²
matrixSize = size(premShellGravity, 1)
for i in range(1, matrixSize)
    shellPos = premShellGravity[i, 1]
    shellGravity = premShellGravity[i, 2]
    if shellPos <= objectPos
        (netAccel += shellGravity/(shellPos)^2)
    end
end
display(netAccel)

NaN

In [ ]:
# Euler's method